In [1]:
import numpy as np
import gymnasium as gym
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

# Environnement Taxi
env = gym.make('Taxi-v3')
n_states = env.observation_space.n
n_actions = env.action_space.n

# Paramètres Monte Carlo optimisés
gamma = 0.95  # Pondération plus faible pour ne pas trop considérer les récompenses futures
epsilon = 1.0  # Exploration maximale au départ
epsilon_min = 0.01  # Valeur minimale d'exploration
epsilon_decay = 0.9995  # Décroissance plus lente pour plus d'exploration
num_episodes = 2000  # Garder les 2000 épisodes
max_steps = 200  # Limite du nombre d'actions par épisode

# Initialisation des variables
Q = np.zeros((n_states, n_actions))  # Q-table initiale
returns_sum = {}
returns_count = {}
success_rates = []
successful_episodes = 0
best_success_rate = 0

# Choix d'action epsilon-greedy
def choose_action(state, epsilon):
    return np.random.randint(n_actions) if np.random.rand() < epsilon else np.argmax(Q[state])

# Générer un épisode
def generate_episode(epsilon):
    episode = []
    state, _ = env.reset()
    for _ in range(max_steps):
        action = choose_action(state, epsilon)
        next_state, reward, terminated, truncated, _ = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if terminated or truncated:
            return episode, reward == 20  # Réussi si la récompense est 20
    return episode, False

# Entraînement avec Monte Carlo First-Visit
for episode_idx in range(1, num_episodes + 1):
    episode, success = generate_episode(epsilon)
    if success:
        successful_episodes += 1

    # Mise à jour de la Q-table avec First-Visit Monte Carlo
    G = 0
    visited = set()
    for t in reversed(range(len(episode))):
        state, action, reward = episode[t]
        G = reward + gamma * G
        if (state, action) not in visited:
            visited.add((state, action))
            returns_sum[(state, action)] = returns_sum.get((state, action), 0) + G
            returns_count[(state, action)] = returns_count.get((state, action), 0) + 1
            Q[state, action] = returns_sum[(state, action)] / returns_count[(state, action)]  # Mise à jour de la Q-table

    # Mise à jour de epsilon pour favoriser l'exploitation
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # Évaluer tous les 100 épisodes
    if episode_idx % 100 == 0:
        rate = (successful_episodes / episode_idx) * 100
        success_rates.append(rate)

        print(f"📊 Épisode {episode_idx:>5} | Taux de réussite: {rate:>5.2f}%")

        # Si meilleure performance trouvée
        if rate > best_success_rate:
            best_success_rate = rate

env.close()


📊 Épisode   100 | Taux de réussite:  9.00%
📊 Épisode   200 | Taux de réussite:  9.50%
📊 Épisode   300 | Taux de réussite: 11.67%
📊 Épisode   400 | Taux de réussite: 12.25%
📊 Épisode   500 | Taux de réussite: 16.00%
📊 Épisode   600 | Taux de réussite: 19.00%
📊 Épisode   700 | Taux de réussite: 21.29%
📊 Épisode   800 | Taux de réussite: 23.88%
📊 Épisode   900 | Taux de réussite: 25.67%
📊 Épisode  1000 | Taux de réussite: 28.00%
📊 Épisode  1100 | Taux de réussite: 29.91%
📊 Épisode  1200 | Taux de réussite: 31.50%
📊 Épisode  1300 | Taux de réussite: 33.38%
📊 Épisode  1400 | Taux de réussite: 34.93%
📊 Épisode  1500 | Taux de réussite: 36.07%
📊 Épisode  1600 | Taux de réussite: 37.25%
📊 Épisode  1700 | Taux de réussite: 38.29%
📊 Épisode  1800 | Taux de réussite: 38.94%
📊 Épisode  1900 | Taux de réussite: 39.21%
📊 Épisode  2000 | Taux de réussite: 39.60%
